In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './Preprocessing')
import numpy as np
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from imgDataGenerator import *
from preprocessing import *
import tensorflow as tf
import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D,MaxPooling3D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU, ConvLSTM2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
from keras.optimizers import Adam
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
#Note for Test, Training, and Validation folders, they all need to have same categories
parentPath='/home/livelab/Desktop/NewResizedBinned'
checkAndGeneratePath(parentPath)
testPath='/home/livelab/Desktop/NewResizedBinned/Test'
trainPath='/home/livelab/Desktop/NewResizedBinned/Training'
validPath='/home/livelab/Desktop/NewResizedBinned/Validation'

trainingGen=imgDataGenerator(trainPath)
validGen=imgDataGenerator(validPath)

Length Check on Folder Succeed


In [3]:
inputS  = Input(shape=(None, 256, 256, 3))
    
first_ConvLSTM = ConvLSTM2D(filters=20, kernel_size=(3, 3)
                       , data_format='channels_last'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                       , padding='same', return_sequences=True)(inputS)
first_BatchNormalization = BatchNormalization()(first_ConvLSTM)
first_Pooling = MaxPooling3D(pool_size=(2, 2, 2), padding='same', data_format='channels_last')(first_BatchNormalization)
    
second_ConvLSTM = ConvLSTM2D(filters=15, kernel_size=(3, 3)
                        , data_format='channels_last'
                        , padding='same', return_sequences=True)(first_Pooling)
second_BatchNormalization = BatchNormalization()(second_ConvLSTM)
second_Pooling = MaxPooling3D(pool_size=( 3, 3,3), padding='same', data_format='channels_last')(second_BatchNormalization)
third_ConvLSTM = ConvLSTM2D(filters=10, kernel_size=(3, 3)
                        , data_format='channels_last'
                        , padding='same', return_sequences=False)(second_Pooling)
third_BatchNormalization = BatchNormalization()(third_ConvLSTM)
thid_Pooling = MaxPooling2D(pool_size=( 3, 3), padding='same', data_format='channels_last')(third_BatchNormalization)
intermediateOut = Flatten()(thid_Pooling)
intermediateOut = Dense(32)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(24, activation='softmax')(intermediateOut)
optmr = Adam(lr=0.0005, decay = 5e-4)
model_final = Model(input = inputS, output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])   

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 256, 256, 20 16640     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256, 256, 20 80        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, None, 128, 128, 20 0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 128, 128, 15 18960     
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 128, 128, 15 60        
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, None, 43, 43, 15)  0         
__________

In [5]:
filepath = "CLSTM-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]

In [6]:
with tf.device('GPU:0'):
    model_final.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    callbacks=callbacks_list,
                    use_multiprocessing=True,
                    epochs = 40,
                    workers=6)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
1286/1286 [==============================] - 280s 217ms/step - loss: 3.1760 - acc: 0.0529 - val_loss: 14.4103 - val_acc: 0.0392

Epoch 00001: val_acc improved from -inf to 0.03922, saving model to CLSTM-01-0.04.hdf5
Epoch 2/40
1286/1286 [==============================] - 276s 215ms/step - loss: 3.1699 - acc: 0.0684 - val_loss: 14.8617 - val_acc: 0.0392

Epoch 00002: val_acc did not improve from 0.03922
Epoch 3/40
1286/1286 [==============================] - 277s 215ms/step - loss: 3.1662 - acc: 0.0677 - val_loss: 14.7290 - val_acc: 0.0343

Epoch 00003: val_acc did not improve from 0.03922
Epoch 4/40
1286/1286 [==============================] - 278s 216ms/step - loss: 3.1627 - acc: 0.0715 - val_loss: 14.5779 - val_acc: 0.0392

Epoch 00004: val_acc did not improve from 0.03922
Epoch 5/40
1286/1286 [==============================] - 276s 215ms/step - loss: 3.1599 - acc: 0.0700 - val_loss: 14.8666 - val_acc: 0.0294

Epoch 00005: v

1286/1286 [==============================] - 274s 213ms/step - loss: 3.1537 - acc: 0.0708 - val_loss: 14.6179 - val_acc: 0.0490

Epoch 00040: val_acc did not improve from 0.06863
